In [11]:
import pandas as pd

In [12]:
df = pd.read_csv('../data/raw/all_years.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12424 entries, 0 to 12423
Data columns (total 52 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   fips               12424 non-null  int64  
 1   totalvotes         12424 non-null  int64  
 2   mode               12424 non-null  object 
 3   dem                12424 non-null  int64  
 4   repub              12424 non-null  int64  
 5   other              12424 non-null  int64  
 6   pop_tot            12424 non-null  int64  
 7   pop_male           12424 non-null  int64  
 8   pop_female         12424 non-null  int64  
 9   hispanic           12424 non-null  int64  
 10  labor_total        12424 non-null  float64
 11  labor_armed        12424 non-null  float64
 12  labor_employed     12424 non-null  float64
 13  labor_unemployed   12424 non-null  float64
 14  income_median      12424 non-null  float64
 15  income_percapita   12424 non-null  float64
 16  income_poverty     124

In [13]:
df = df.drop(columns=['mode'])

In [14]:
# Convert float columns to int (except CENSUSAREA and pop_density)
float_columns = df.select_dtypes(include=['float64', 'float32']).columns.tolist()

# Exclude CENSUSAREA and pop_density
columns_to_convert = [col for col in float_columns if col not in ['CENSUSAREA', 'pop_density']]

# Convert each column to int (this will truncate decimal parts)
for col in columns_to_convert:
    original_dtype = df[col].dtype
    df[col] = df[col].astype(int)
    print(f"Converted {col}: {original_dtype} -> {df[col].dtype}")

print(f"\nConversion complete! {len(columns_to_convert)} columns converted to int.")

Converted labor_total: float64 -> int64
Converted labor_armed: float64 -> int64
Converted labor_employed: float64 -> int64
Converted labor_unemployed: float64 -> int64
Converted income_median: float64 -> int64
Converted income_percapita: float64 -> int64
Converted income_poverty: float64 -> int64
Converted native_yes: float64 -> int64
Converted native_no: float64 -> int64
Converted income_10: float64 -> int64
Converted income_10-15: float64 -> int64
Converted income_15-25: float64 -> int64
Converted income_25: float64 -> int64

Conversion complete! 13 columns converted to int.


In [15]:
# Add new columns for population analysis
df['18plus'] = df['edu_low'] + df['edu_mid'] + df['edu_high']
df['underage'] = df['pop_tot'] - df['18plus']
df['nonvoter'] = df['18plus'] - df['totalvotes']

# Identify problematic FIPS codes
problematic_underage_fips = df[df['underage'] < 0]['fips'].tolist()
problematic_nonvoter_fips = df[df['nonvoter'] < 0]['fips'].tolist()

print(f"Problematic FIPS codes where underage is negative: {len(problematic_underage_fips)}")
print(f"Problematic FIPS codes where nonvoter is negative: {len(problematic_nonvoter_fips)}")

# Drop rows with problematic FIPS codes
df = df[~df['fips'].isin(problematic_underage_fips + problematic_nonvoter_fips)].copy()

print(f"Cleaned dataframe shape: {df.shape}")
print(df.groupby('year').size())

df.info()

Problematic FIPS codes where underage is negative: 0
Problematic FIPS codes where nonvoter is negative: 26
Cleaned dataframe shape: (12368, 54)
year
2008    3092
2012    3092
2016    3092
2020    3092
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 12368 entries, 0 to 12423
Data columns (total 54 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   fips               12368 non-null  int64  
 1   totalvotes         12368 non-null  int64  
 2   dem                12368 non-null  int64  
 3   repub              12368 non-null  int64  
 4   other              12368 non-null  int64  
 5   pop_tot            12368 non-null  int64  
 6   pop_male           12368 non-null  int64  
 7   pop_female         12368 non-null  int64  
 8   hispanic           12368 non-null  int64  
 9   labor_total        12368 non-null  int64  
 10  labor_armed        12368 non-null  int64  
 11  labor_employed     12368 non-null  int64  
 12  labor

In [16]:
# Create column descriptions
column_descriptions = {
    'fips': 'Federal Information Processing Standards county code identifier',
    'totalvotes': 'Total number of votes cast in the county',
    'dem': 'Number of votes for Democratic candidate',
    'repub': 'Number of votes for Republican candidate',
    'other': 'Number of votes for other candidates',
    'pop_tot': 'Total population of the county',
    'pop_male': 'Male population of the county',
    'pop_female': 'Female population of the county',
    'hispanic': 'Number of Hispanic residents in the county',
    'labor_total': 'Total labor force in the county',
    'labor_armed': 'Armed forces in the labor force',
    'labor_employed': 'Number of employed persons in the county',
    'labor_unemployed': 'Number of unemployed persons in the county',
    'income_median': 'Median household income in the county',
    'income_percapita': 'Per capita income in the county',
    'income_poverty': 'Population below poverty line',
    'native_yes': 'Number of native-born residents',
    'native_no': 'Number of foreign-born residents',
    'houses_tot': 'Total number of housing units',
    'income_10': 'Number of households with income less than $10,000',
    'income_10-15': 'Number of households with income $10,000-$14,999',
    'income_15-25': 'Number of households with income $15,000-$24,999',
    'income_25': 'Number of households with income $25,000 or more',
    'marital_single': 'Number of single (never married) residents',
    'marital_married': 'Number of married residents',
    'marital_sepdiv': 'Number of separated or divorced residents',
    'marital_widow': 'Number of widowed residents',
    'edu_low': 'Number of residents with low education (less than high school)',
    'edu_mid': 'Number of residents with medium education (high school or some college)',
    'edu_high': 'Number of residents with high education (bachelor\'s degree or higher)',
    'edu_low_age_18': 'Number of residents 18-44 with low education',
    'edu_low_age_45': 'Number of residents 45-64 with low education',
    'edu_low_age_65': 'Number of residents 65 and over with low education',
    'edu_mid_age_18': 'Number of residents 18-44 with medium education',
    'edu_mid_age_45': 'Number of residents 45-64 with medium education',
    'edu_mid_age_65': 'Number of residents 65 and over with medium education',
    'edu_high_age_18': 'Number of residents 18-44 with high education',
    'edu_high_age_45': 'Number of residents 45-64 with high education',
    'edu_high_age_65': 'Number of residents 65 and over with high education',
    'race_wht_total': 'Total white population',
    'race_wht_male': 'White male population',
    'race_wht_female': 'White female population',
    'race_blk_total': 'Total black population',
    'race_blk_male': 'Black male population',
    'race_blk_female': 'Black female population',
    'race_other_total': 'Total population of other races',
    'race_other_male': 'Male population of other races',
    'race_other_female': 'Female population of other races',
    'CENSUSAREA': 'Census area in square miles',
    'pop_density': 'Population density (persons per square mile)',
    'year': 'Election year',
    '18plus': 'Number of residents aged 18 and over (calculated from education data)',
    'underage': 'Number of residents under 18 years old (calculated as total population minus 18plus)',
    'nonvoter': 'Number of eligible voters who did not vote (calculated as 18plus minus totalvotes)'
}

# Define units for each column
column_units = {
    'fips': 'id',
    'totalvotes': 'persons',
    'dem': 'persons',
    'repub': 'persons',
    'other': 'persons',
    'pop_tot': 'persons',
    'pop_male': 'persons',
    'pop_female': 'persons',
    'hispanic': 'persons',
    'labor_total': 'persons',
    'labor_armed': 'persons',
    'labor_employed': 'persons',
    'labor_unemployed': 'persons',
    'income_median': 'dollars',
    'income_percapita': 'dollars_per_person',
    'income_poverty': 'persons',
    'native_yes': 'persons',
    'native_no': 'persons',
    'houses_tot': 'households',
    'income_10': 'households',
    'income_10-15': 'households',
    'income_15-25': 'households',
    'income_25': 'households',
    'marital_single': 'persons',
    'marital_married': 'persons',
    'marital_sepdiv': 'persons',
    'marital_widow': 'persons',
    'edu_low': 'persons',
    'edu_mid': 'persons',
    'edu_high': 'persons',
    'edu_low_age_18': 'persons',
    'edu_low_age_45': 'persons',
    'edu_low_age_65': 'persons',
    'edu_mid_age_18': 'persons',
    'edu_mid_age_45': 'persons',
    'edu_mid_age_65': 'persons',
    'edu_high_age_18': 'persons',
    'edu_high_age_45': 'persons',
    'edu_high_age_65': 'persons',
    'race_wht_total': 'persons',
    'race_wht_male': 'persons',
    'race_wht_female': 'persons',
    'race_blk_total': 'persons',
    'race_blk_male': 'persons',
    'race_blk_female': 'persons',
    'race_other_total': 'persons',
    'race_other_male': 'persons',
    'race_other_female': 'persons',
    'CENSUSAREA': 'area',
    'pop_density': 'persons_per_area',
    'year': 'id',
    '18plus': 'persons',
    'underage': 'persons',
    'nonvoter': 'persons'
}

# Define roles for each column
id_columns = ['fips', 'year']
target_columns = ['dem', 'repub', 'other', 'nonvoter']
feature_columns = [col for col in column_descriptions.keys() if col not in id_columns + target_columns]

column_roles = {}
for col in column_descriptions.keys():
    if col in id_columns:
        column_roles[col] = 'id'
    elif col in target_columns:
        column_roles[col] = 'target'
    else:
        column_roles[col] = 'feature'

# Create DataFrame with column information

column_info = pd.DataFrame({
    'name': list(column_descriptions.keys()),
    'description': list(column_descriptions.values()),
    'unit': [column_units[col] for col in column_descriptions.keys()],
    'role': [column_roles[col] for col in column_descriptions.keys()]
})

# Save to CSV
column_info.to_csv('../data/raw/column_descriptions.csv', index=False)
print("Column descriptions saved to CSV with shape:", column_info.shape)
print("\nRole distribution:")
print(column_info['role'].value_counts())
print("\nUnit distribution:")
print(column_info['unit'].value_counts())

Column descriptions saved to CSV with shape: (54, 4)

Role distribution:
role
feature    48
target      4
id          2
Name: count, dtype: int64

Unit distribution:
unit
persons               43
households             5
id                     2
dollars                1
dollars_per_person     1
area                   1
persons_per_area       1
Name: count, dtype: int64


In [17]:
# Read column descriptions
column_info = pd.read_csv('../data/raw/column_descriptions.csv')

# Group columns by type
id_cols = column_info[column_info['role'] == 'id']['name'].tolist()
target_cols = column_info[column_info['role'] == 'target']['name'].tolist()
person_cols = column_info[column_info['unit'] == 'persons']['name'].tolist()
house_cols = column_info[column_info['unit'] == 'households']['name'].tolist()

# Get person features (excluding targets)
person_features = [col for col in person_cols if col not in target_cols]

# Get remaining features
misc_cols = column_info[~column_info['unit'].isin(['persons', 'households', 'id'])]['name'].tolist()

# Create normalized dataframe
df_normalized = pd.DataFrame(index=df.index)

# 1. Add ID columns
df_normalized[id_cols] = df[id_cols]

# 2. Add normalized person features (P(feature|C))
df_normalized[person_features] = df[person_features].div(df['pop_tot'], axis=0)

# 3. Add normalized household features (P(feature|C))
df_normalized[house_cols] = df[house_cols].div(df['houses_tot'], axis=0)

# 4. Add remaining features
df_normalized[misc_cols] = df[misc_cols]

# 5. Add normalized targets (P(party|C)) - at the end
df_normalized[target_cols] = df[target_cols].div(df['pop_tot'], axis=0)

# 6. Drop totalvotes, pop_tot, houses_tot as they are redundant now
df_normalized = df_normalized.drop(columns=['totalvotes', 'pop_tot', 'houses_tot'])

print(f"Normalized dataframe shape: {df_normalized.shape}")
df_normalized.info()

Normalized dataframe shape: (12368, 51)
<class 'pandas.core.frame.DataFrame'>
Index: 12368 entries, 0 to 12423
Data columns (total 51 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   fips               12368 non-null  int64  
 1   year               12368 non-null  int64  
 2   pop_male           12368 non-null  float64
 3   pop_female         12368 non-null  float64
 4   hispanic           12368 non-null  float64
 5   labor_total        12368 non-null  float64
 6   labor_armed        12368 non-null  float64
 7   labor_employed     12368 non-null  float64
 8   labor_unemployed   12368 non-null  float64
 9   income_poverty     12368 non-null  float64
 10  native_yes         12368 non-null  float64
 11  native_no          12368 non-null  float64
 12  marital_single     12368 non-null  float64
 13  marital_married    12368 non-null  float64
 14  marital_sepdiv     12368 non-null  float64
 15  marital_widow      12368 non-null  

In [18]:
# Define proper descriptions for normalized columns
normalized_descriptions = {
    # ID columns (unchanged)
    'fips': 'Federal Information Processing Standards county code identifier',
    'year': 'Election year',
    
    # Target columns (probabilities)
    'dem': 'Proportion of county population voting Democratic',
    'repub': 'Proportion of county population voting Republican', 
    'other': 'Proportion of county population voting for other candidates',
    'nonvoter': 'Proportion of eligible voters who did not vote',
    
    # Demographic features (population proportions)
    'pop_male': 'Proportion of county population that is male',
    'pop_female': 'Proportion of county population that is female',
    'hispanic': 'Proportion of county population that is Hispanic',
    '18plus': 'Proportion of county population aged 18 and over',
    'underage': 'Proportion of county population under 18 years old',
    
    # Labor force features
    'labor_total': 'Proportion of county population in the labor force',
    'labor_armed': 'Proportion of labor force in armed forces',
    'labor_employed': 'Proportion of county population that is employed',
    'labor_unemployed': 'Proportion of county population that is unemployed',
    
    # Income features
    'income_poverty': 'Proportion of county population below poverty line',
    'income_percapita': 'Per capita income in dollars',
    'income_median': 'Median household income in dollars',
    
    # Immigration features
    'native_yes': 'Proportion of county population that is native-born',
    'native_no': 'Proportion of county population that is foreign-born',
    
    # Housing features (household proportions)
    'income_10': 'Proportion of households with income less than $10,000',
    'income_10-15': 'Proportion of households with income $10,000-$14,999',
    'income_15-25': 'Proportion of households with income $15,000-$24,999',
    'income_25': 'Proportion of households with income $25,000 or more',
    
    # Marital status features
    'marital_single': 'Proportion of county population that is single',
    'marital_married': 'Proportion of county population that is married',
    'marital_sepdiv': 'Proportion of county population that is separated/divorced',
    'marital_widow': 'Proportion of county population that is widowed',
    
    # Education features
    'edu_low': 'Proportion of county population with less than high school education',
    'edu_mid': 'Proportion of county population with high school or some college education',
    'edu_high': 'Proportion of county population with bachelor\'s degree or higher',
    
    # Age-specific education features
    'edu_low_age_18': 'Proportion of 18-44 year olds with less than high school education',
    'edu_low_age_45': 'Proportion of 45-64 year olds with less than high school education',
    'edu_low_age_65': 'Proportion of 65+ year olds with less than high school education',
    'edu_mid_age_18': 'Proportion of 18-44 year olds with high school or some college education',
    'edu_mid_age_45': 'Proportion of 45-64 year olds with high school or some college education',
    'edu_mid_age_65': 'Proportion of 65+ year olds with high school or some college education',
    'edu_high_age_18': 'Proportion of 18-44 year olds with bachelor\'s degree or higher',
    'edu_high_age_45': 'Proportion of 45-64 year olds with bachelor\'s degree or higher',
    'edu_high_age_65': 'Proportion of 65+ year olds with bachelor\'s degree or higher',
    
    # Racial demographics
    'race_wht_total': 'Proportion of county population that is white',
    'race_wht_male': 'Proportion of county population that is white male',
    'race_wht_female': 'Proportion of county population that is white female',
    'race_blk_total': 'Proportion of county population that is black',
    'race_blk_male': 'Proportion of county population that is black male',
    'race_blk_female': 'Proportion of county population that is black female',
    'race_other_total': 'Proportion of county population of other races',
    'race_other_male': 'Proportion of county population that is male of other races',
    'race_other_female': 'Proportion of county population that is female of other races',
    
    # Geographic features (unchanged)
    'CENSUSAREA': 'Census area in square miles',
    'pop_density': 'Population density (persons per square mile)'
}

normalized_column_info = pd.DataFrame({
    'name': list(normalized_descriptions.keys()),
    'description': list(normalized_descriptions.values()),
    'unit': [column_units[col] for col in normalized_descriptions.keys()],
    'role': [column_roles[col] for col in normalized_descriptions.keys()]
})

In [19]:
df_normalized.to_csv('../data/cleaned/dataset.csv', index=False)
normalized_column_info.to_csv('../data/cleaned/column_descriptions.csv', index=False)